In [37]:
import pandas as pd
df_rotatividade = pd.read_csv('df_rotatividade_filtrado.csv')
df_microdados = pd.read_csv('df_microdados_filtrado.csv')

In [38]:
# Primeiro, vamos definir as colunas que queremos adicionar ao dataframe df_rotatividade
colunas_desejadas = ["Nome Escola", "IN_INTERNET", "TP_DEPENDENCIA", "IN_ENERGIA_REDE_PUBLICA", "IN_ESGOTO_REDE_PUBLICA", "IN_LIXO_SERVICO_COLETA", "IN_LABORATORIO_CIENCIAS", "IN_LABORATORIO_INFORMATICA" ]

# Em seguida, criamos um novo dataframe que contém apenas as colunas desejadas
df_microdados_desejados = df_microdados[colunas_desejadas]

# Por fim, usamos o método merge para unir os dois dataframes com base na coluna 'Nome Escola'
df_rotatividade = df_rotatividade.merge(df_microdados_desejados, on='Nome Escola')


In [16]:
df_rotatividade.head()


,Nome Escola,Pontuação IRD,IN_INTERNET,TP_LOCALIZACAO,TP_DEPENDENCIA,IN_AGUA_FILTRADA,IN_ENERGIA_REDE_PUBLICA,IN_ESGOTO_REDE_PUBLICA,IN_LIXO_SERVICO_COLETA
0,JOAO XXIII,3.135485,1.0,1,2,NaN,1.0,1.0,1.0
1,JOAO XXIII,3.054475,1.0,1,2,NaN,1.0,1.0,1.0
2,CARLOS GOMES,2.789169,1.0,1,2,NaN,1.0,1.0,1.0
3,CARLOS GOMES,2.688652,1.0,1,2,NaN,1.0,1.0,1.0
4,EMEI MONTEIRO LOBATO,4.239130,1.0,1,3,NaN,1.0,1.0,1.0


In [17]:
!pip install statsmodels


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\Alan Matheus\Desktop\pesquisas\env\Scripts\python.exe -m pip install --upgrade pip


In [53]:
print(df_rotatividade['TP_DEPENDENCIA'].value_counts())
df_rotatividade['EH_ESTADUAL'] = df_rotatividade['TP_DEPENDENCIA'].apply(lambda x: 1 if x == 2 else 0)
df_rotatividade['EH_MUNICIPAL'] = df_rotatividade['TP_DEPENDENCIA'].apply(lambda x: 1 if x == 3 else 0)
df_rotatividade['EH_RURAL'] = df_rotatividade['TP_LOCALIZACAO'].apply(lambda x: 1 if x == 2 else 0)
df_rotatividade['EH_URBANA'] = df_rotatividade['TP_LOCALIZACAO'].apply(lambda x: 1 if x == 1 else 0)

df_rotatividade = df_rotatividade.reset_index(drop=True).dropna()
X = df_rotatividade[["IN_INTERNET", "EH_ESTADUAL","EH_MUNICIPAL", "IN_ENERGIA_REDE_PUBLICA", "IN_ESGOTO_REDE_PUBLICA", "IN_LIXO_SERVICO_COLETA"]]
y = df_rotatividade["Pontuação IRD"]
X.head()

TP_DEPENDENCIA
3    1505
2    1169
1       3
Name: count, dtype: int64


,IN_INTERNET,EH_ESTADUAL,EH_MUNICIPAL,IN_ENERGIA_REDE_PUBLICA,IN_ESGOTO_REDE_PUBLICA,IN_LIXO_SERVICO_COLETA
0,1.0,1,0,1.0,1.0,1.0
1,1.0,1,0,1.0,1.0,1.0
2,1.0,1,0,1.0,1.0,1.0
3,1.0,1,0,1.0,1.0,1.0
4,1.0,0,1,1.0,1.0,1.0


In [54]:
import statsmodels.api as sm



# Adiciona uma constante ao X para representar o coeficiente linear da regressão
X = sm.add_constant(X)

# Cria um modelo de regressão OLS (Ordinary Least Squares)
model = sm.OLS(y, X)

# Ajusta o modelo aos dados
result = model.fit()

# Imprime um resumo dos resultados
print(result.summary())


                            OLS Regression Results                            
Dep. Variable:          Pontuação IRD   R-squared:                       0.351
Model:                            OLS   Adj. R-squared:                  0.350
Method:                 Least Squares   F-statistic:                     289.2
Date:                Thu, 01 Jun 2023   Prob (F-statistic):          8.83e-248
Time:                        14:11:48   Log-Likelihood:                -1741.4
No. Observations:                2677   AIC:                             3495.
Df Residuals:                    2671   BIC:                             3530.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
IN_INTERNET               